In [1]:
import _init_paths

import torch
import os
from model.faster_rcnn.vgg16 import vgg16
from model.faster_rcnn.resnet import resnet
import torch.nn as nn
from model.utils.config import cfg
from model.utils.net_utils import save_checkpoint
import FedUtils

In [2]:
class Arguments():
    def __init__(self):
        self.class_agnostic =False
        self.epochs = 3
        self.lr = 0.001
        self.lr_decay_step = 4
        self.dataset = 'KAIST'
        self.net = 'vgg16'
        self.batch_size = 24
        self.nw = 2
        self.cuda = True
        #self.round = 10
        self.mGPUs=True
        self.optimizer ="sgd"
        self.k=5
        #self.device = th.device("cpu")
        
args = Arguments()

## load model

In [10]:
def load_models( round_i):
    model_path = ['models/vgg16/KAIST/'+sub_folder+'/faster_rcnn_KAIST_campus_'+str(round_i)+'_3_718.pth',
        'models/vgg16/KAIST/'+sub_folder+'/faster_rcnn_KAIST_road_'+str(round_i)+'_3_513.pth',
        'models/vgg16/KAIST/'+sub_folder+'/faster_rcnn_KAIST_downtown_'+str(round_i)+'_3_566.pth']
    
    parties = len(model_path)


    model_list=[None] * parties
    for i in range(parties):
        model_list[i],optimizer, start_round = FedUtils.load_model(imdb_classes,model_path[i], args,cfg)
    return model_list, optimizer

In [5]:
def save_avgModel(saved_model, optimizer, output_dir, round_i, args, cfg):

    save_name = os.path.join(output_dir, 'faster_rcnn_KAIST_AVG_{}.pth'.format(round_i))
    print('save model to {}'.format(save_name))
    save_checkpoint({
      #'session': 1,
      'round': round_i,
      'model':  saved_model.module.state_dict() if args.mGPUs else saved_model.state_dict(), 
      'optimizer': optimizer.state_dict(),
      'pooling_mode': cfg.POOLING_MODE,
      'class_agnostic': args.class_agnostic,
    }, save_name)

In [8]:


sub_folder = 'FedPer'

imdb_classes =  ('__background__',  # always index 0
                          'person',
                          'people','cyclist'
                         )



In [12]:
ratio_list = [0.33333,0.33333,0.33333]

output_dir = "models/" + args.net + "/" + args.dataset + "/" + sub_folder


for round_i in range(1,10+1):
    print(round_i)
    model_list, optimizer = load_models( round_i)
    model_avg = FedUtils.avgWeight(model_list,ratio_list)
    save_avgModel(model_avg[0], optimizer, output_dir, round_i, args, cfg)

1
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
save model to models/vgg16/KAIST/FedPer/faster_rcnn_KAIST_AVG_1.pth
2
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
save model to models/vgg16/KAIST/FedPer/faster_rcnn_KAIST_AVG_2.pth
3
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
save model to models/vgg16/KAIST/FedPer/faster_rcnn_KAIST_AVG_3.pth
4
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained

In [13]:
for model in model_list:
    count=0
    for key in model.module.RCNN_base.state_dict():
        print(key)
        print(model.module.RCNN_base.state_dict()[key])
        if count>2:
            break
        count+=1

0.weight
tensor([[[[ 4.2947e-01,  3.7346e-01, -6.1359e-02],
          [ 2.7477e-01,  3.8680e-02, -3.6722e-01],
          [-5.7468e-02, -2.6225e-01, -3.5009e-01]],

         [[ 5.5037e-01,  4.4007e-01, -8.1386e-02],
          [ 3.4574e-01,  4.0632e-02, -4.5350e-01],
          [-5.8634e-02, -3.3067e-01, -4.8503e-01]],

         [[ 4.8001e-01,  4.0854e-01, -6.5145e-02],
          [ 3.1047e-01,  5.0202e-02, -4.0338e-01],
          [-5.0871e-02, -2.8522e-01, -4.1851e-01]]],


        [[[ 1.1727e-01,  1.6206e-01,  1.3569e-01],
          [ 1.4835e-01,  2.0230e-01,  1.6169e-01],
          [ 1.2934e-01,  1.7157e-01,  1.3871e-01]],

         [[ 2.0877e-02,  4.7341e-02,  4.1854e-02],
          [ 3.1049e-02,  6.5810e-02,  4.6257e-02],
          [ 3.1678e-02,  5.4710e-02,  4.2319e-02]],

         [[-1.7269e-01, -1.7037e-01, -1.5435e-01],
          [-1.8760e-01, -1.7757e-01, -1.7440e-01],
          [-1.6600e-01, -1.6667e-01, -1.5705e-01]]],


        [[[ 3.4013e-02,  1.7086e-03, -1.1569e-01],
      

In [15]:
ct = 0
for child in model_list[0].children():
    ct += 1
    if ct < 7:
        for param in child.parameters():
            print(param)
            #param.requires_grad = False


Parameter containing:
tensor([[[[-9.6468e-03, -6.9796e-03,  7.6043e-03],
          [ 1.5817e-03,  2.5088e-03, -2.3906e-02],
          [ 6.7706e-03,  1.1092e-02,  9.2745e-03]],

         [[-4.9113e-04,  1.5139e-02, -5.6044e-03],
          [ 3.2027e-04, -3.0482e-03, -1.4502e-02],
          [ 6.0405e-03, -7.9695e-03,  1.4797e-03]],

         [[ 9.6381e-03, -5.9795e-03, -1.8189e-02],
          [-1.7891e-02,  4.9025e-04,  4.4717e-03],
          [-1.6159e-03,  3.9536e-04,  2.0451e-03]],

         ...,

         [[-5.8637e-03, -2.1459e-03,  5.2929e-03],
          [-5.3769e-03,  3.6548e-03,  5.8917e-03],
          [ 9.2719e-03, -3.3455e-03,  4.3174e-03]],

         [[ 6.8835e-03,  5.4372e-03,  3.1679e-04],
          [-2.5302e-03, -3.5710e-03,  5.2753e-03],
          [-2.0050e-03,  1.2269e-02,  2.3463e-03]],

         [[ 1.0209e-03,  1.1126e-02,  1.4257e-02],
          [ 4.6109e-03,  1.1933e-02,  1.5531e-03],
          [ 1.0454e-02,  3.3894e-03,  7.0604e-04]]],


        [[[-8.8856e-03,  1.8182

Parameter containing:
tensor([[[[-0.0104, -0.0044,  0.0041],
          [-0.0140, -0.0136, -0.0061],
          [-0.0093, -0.0127, -0.0095]],

         [[-0.0089, -0.0029, -0.0032],
          [-0.0098, -0.0073, -0.0056],
          [ 0.0023, -0.0036, -0.0009]],

         [[ 0.0148,  0.0235,  0.0248],
          [ 0.0135,  0.0215,  0.0193],
          [-0.0050,  0.0089,  0.0035]],

         ...,

         [[-0.0124, -0.0068, -0.0086],
          [-0.0024,  0.0043,  0.0025],
          [ 0.0133,  0.0277,  0.0232]],

         [[ 0.0015,  0.0095,  0.0112],
          [-0.0080,  0.0060,  0.0091],
          [-0.0051, -0.0017, -0.0021]],

         [[-0.0047, -0.0144, -0.0154],
          [ 0.0029, -0.0105, -0.0101],
          [ 0.0098,  0.0002, -0.0010]]],


        [[[-0.0014, -0.0021, -0.0028],
          [ 0.0047,  0.0265,  0.0051],
          [-0.0044,  0.0087, -0.0033]],

         [[-0.0014,  0.0034,  0.0066],
          [-0.0009,  0.0027,  0.0036],
          [ 0.0057,  0.0054,  0.0031]],

         

Parameter containing:
tensor([ 9.7248e-02, -7.3236e-02,  1.5152e-01,  1.0737e-01, -7.8729e-02,
        -2.4147e-02,  8.4792e-02,  6.2458e-02,  2.8885e-01,  1.5588e-01,
         1.3398e-01,  6.9818e-02, -2.1720e-01,  1.0593e-01,  2.2225e-01,
        -2.8771e-02,  6.4758e-02, -1.3882e-01,  2.0530e-01,  1.1939e-01,
        -4.1830e-02, -1.1128e-02, -8.7132e-02,  7.5546e-02,  2.6412e-01,
         9.7387e-02, -6.5616e-02, -1.3963e-01, -1.6264e-01, -8.7792e-03,
         2.0973e-01,  1.2062e-01,  2.3301e-01,  1.4279e-02,  1.2702e-01,
         4.3296e-02,  1.0756e-01,  2.5599e-02, -7.2263e-02,  5.7015e-02,
         3.5124e-02, -4.4123e-02, -7.6632e-02,  1.8154e-01,  1.8033e-01,
         1.6878e-01,  9.7841e-02,  1.8047e-02, -1.4361e-02,  7.4187e-02,
         9.3959e-02,  1.1945e-01, -1.2610e-01,  3.0745e-01, -9.7852e-02,
         1.5083e-01,  1.1156e-02, -1.3421e-01, -1.3384e-02,  1.4516e-01,
         4.9616e-02,  2.9252e-01,  1.3655e-02, -9.9308e-02,  2.2406e-03,
         1.1245e-01,  1.6715e

In [ ]:
round_i

## resume from avg model